# Field Capacity Finder from the soil moisture

## From the soil moisture data
***
### Data description
- data is being collected from the F70 field.
- soil moisture is in unit of %, **not cleaned**
- this script includes cleaning function
- this script designed to be used in daily basis.
- from the soil moisture monitoring website
- example data is 'Calc_def_test.csv'

- [soil moisture data](https://things.iot.ag.purdue.edu:8080/dashboard/dc56c5a0-ee3e-11ec-b72b-5dd76ca52a2b?publicId=a914a590-ecae-11ec-b72b-5dd76ca52a2b) input data for the script will be downloaded directly
> - soil moisture data is from Purdue AgIT server that collects the field data through LoRaWan network
> - the data should be downloaded from the 'Summary Data Table' tab
> 
- [weather data] input data for the script will be downloaded directly
> - weather data is from Purdue Mesonet server and using ACRE station data
>

For downloading the data
  - Summary data table
  - from date1 to date2
  - in csv form
  - file name does not matter or add today's date at the end of the file name.
  

In [7]:
# // this part is download the data from the website for 2 month.
import requests
import sys
import os
import json
import pandas as pd
from pprint import pprint
import datetime
import pytz
import config
from dateutil.relativedelta import relativedelta

deviceList = []

# ** set the configuration for the request                                                                                  **
config = {
 'username' : 'yang2309@purdue.edu', ### Insert your email address used by AgIT Thingsboard system
 'password': 'dsya2002',  ### Insert your AgIT thingsboard password
 'server' : 'https://things.iot.ag.purdue.edu:8080'
}

# ** defining the function to get the token for the request and setting the header for the request                          **
def getCustomerDevices(custID, textSearch=None):
    parameters = {        
        'pageSize': 1000,
        'page': 0,                
    }
    att_parms = {
        'keys': 'dev_eui'
    }
    if(textSearch):
        parameters.update({'textSearch': textSearch})
    responseList = requests.get(f"{config['server']}/api/customer/{custID}/devices", headers=TBheaders,params= parameters).json()
    #pprint(responseList)
    list = []
    for dev in responseList['data']:
        #pprint(dev)
        #print('------------------------------------------------------------------------------------------')
        #'id': {'entityType': 'DEVICE', 'id': 'd49153a0-c868-11eb-95d8-09d06ef6a9a5'},
        url = f"{config['server']}/api/plugins/telemetry/DEVICE/{dev['id']['id']}/values/attributes"
        deviceResp = requests.get(url, headers=TBheaders,params= att_parms).json()
        #print('------------------------------------------------------------------------------------------')
        list.append([dev['id']['id'],dev['name'],deviceResp[0]['value']])
    return list
        

def login(url, username, password):
    # Log into ThingsBoard
    return requests.post(f"{url}/api/auth/login", json={
        "username": username,
        "password": password
    }).json()['token']

def get_keys(device):
    return requests.get(f"{config['server']}/api/plugins/telemetry/DEVICE/{device}/keys/timeseries",
                 headers=TBheaders).json()
def get_data_chunk(url, token, device, key, start, stop, limit):
    #print([url, device, key, start, stop, limit])
    return requests.get(f"{url}/api/plugins/telemetry/DEVICE/{device}/values/timeseries",
             headers=TBheaders,
            params= {
                'keys': key,
                'startTs': start,
                'endTs': stop,
                'limit': limit,
                'agg': 'NONE'
            }).json()

def get_data(url, token, device, key, start, stop):
    global totalLength
    p = pd.DataFrame()
    
    # You have to request data backwards in time ...
    while start < stop:
        data = get_data_chunk(url, token, device[0], key, start, stop, 100000)
        #print(data)
        if key not in data:
            break;
        
        #print(f"{key}: Loaded {len(data[key])} points")
        t = pd.DataFrame.from_records(data[key])
        #t['Timestamp'] = t['ts']
        #pprint(t['ts'])
        t['ts'] = (pd.to_datetime(t['ts'],unit='ms'))        
        t.set_index('ts', inplace=True)
        
        t.rename(columns={'value': key}, inplace=True)
        p = p._append(t)

        # Update "new" stop time
        stop = data[key][-1]['ts'] - 1
    totalLength += len(p)
    #print(f"Total Length: {totalLength}")
    return p

def outputCSV(devices):
    global totalLength
    final_df = pd.DataFrame()
    for device in devices:
        #print(f"Downloading DEVICE: {device[0]} data");
        #print(device)
        p = pd.DataFrame()
        for key in keys:
            #print(f"info: Pulling {key}...");
            tempin = get_data(config['server'], token, device, key, startTS, endTS)            
            if(len(tempin)>0):                
                p = pd.concat([p,tempin], axis=1)
        p['Entity Name'] = device[1]
        p['dev_eui'] = device[2]
        p.reset_index(drop=False)
        #p_new_index = p.assign(**{'Timestamp': p.index})        
        if(len(p)):
            final_df = pd.concat([final_df,p])
        
    # Create Time Strings
    # Convert to nanoseconds for pandas.to_datetime
    start_timestamp_ns = startTS * 1000000
    end_timestamp_ns = endTS * 1000000
    
    # Convert timestamp to datetime object
    start_dt = pd.to_datetime(start_timestamp_ns, unit='ns')
    end_dt = pd.to_datetime(end_timestamp_ns, unit='ns')
    
    # Format datetime string as yyyy-mm-dd-HH-MM
    start_formatted_string = start_dt.strftime('%Y-%m-%d-%H-%M')
    end_formatted_string = end_dt.strftime('%Y-%m-%d-%H-%M')
    df_order = ["Entity Name","data_soil_moisture1","data_soil_moisture2","data_soil_moisture3","data_soil_moisture4","data_tem1","data_tem2","data_tem3","data_tem4","data_tem5","data_tem6","data_tem7","dev_eui"]
    final_df = final_df.reindex(columns=df_order)
    final_df1 = final_df.sort_values(by='ts')
    
    # Get current time
    now = datetime.datetime.now()
    
    # Format time string (hours and minutes)
    formatted_time = now.strftime("%H-%M")
    final_df1.to_csv(f"./Raw_data/data-{end_formatted_string}.csv")
    print("Done.")

def getDeviceCredentialsByDeviceId(deviceID = 0):
    url = config['server']+'/device/'+deviceID+'/credentials'
    resp = requests.get(url,headers=TBheaders)
    responseList = resp.json()
    #pprint(responseList)
    return responseList['credentialsID']

def getDeviceServerAttributes(deviceID = 0):
    if deviceID == 0:
        while(deviceID == 0):
            try:
                deviceID = input("Enter device ID: ")
            except:
                print("Invalid DeviceID")
    url = config['server']+'/plugins/telemetry/DEVICE/'+deviceID+'/values/attributes'
    #pprint(url)
    #pprint(TBheaders)
    xresp = requests.get(url,headers=TBheaders)
    #pprint(xresp)
    #pprint(resp.content())
    #print(xresp.text())
    responseList = xresp.json()
    #pprint(responseList)
    #return responseList['credentialsID']


# ** getting token for the request                                                                                         **
print("Server: ",config['server'])
token = login(config['server'], config['username'], config['password']);
print(f"Token: {token}")
TBheaders={ 'Accept': '*/*', 'X-Authorization': f"Bearer {token}" }



# Create a datetime object representing the local date and time
# Year, Month, Day, Hour, Minute
today_dt = datetime.datetime.now()
start = datetime.datetime.now()+ relativedelta(months=-2)

start_dt = datetime.datetime(start.year, start.month, start.day, 18, 0)
end_dt = datetime.datetime(today_dt.year, today_dt.month, today_dt.day, 6, 00)
print (start_dt, end_dt)

# Convert to a specific time zone (e.g., UTC)
start_tz_utc = pytz.timezone("UTC")
start_dt_utc = start_tz_utc.localize(start_dt)
end_tz_utc = pytz.timezone("UTC")
end_dt_utc = end_tz_utc.localize(end_dt)

# Extract the Unix timestamp
startTS = int(start_dt_utc.timestamp())*1000
endTS = int(end_dt_utc.timestamp())*1000

# Use for relative time frames
#startTS = int((datetime.now() - timedelta(days=30)  - datetime(1970, 1, 1)).total_seconds() * 1000) # 30 days ago
#endTS = int((datetime.datetime.utcnow() - datetime.datetime(1970, 1, 1)).total_seconds() * 1000) # now

print(startTS, endTS)



# ** customer ID for the request                                                                                            **
# getCustomerDevices(custID, textSearch=None):
# 7576b020-ecae-11ec-b72b-5dd76ca52a2b = Cherkhauer Customer ID
# ABE-DRAGINO-GROPOINT-CHERKHAUER = Devices with names beginning with "ABE-DRAGINO-GROPOINT-CHERKHAUER"
devices = getCustomerDevices("7576b020-ecae-11ec-b72b-5dd76ca52a2b","ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE")
pprint(devices)

totalLength = 0
# keys to retrieve
#keys = ["data_TempC_SHT","data_Hum_SHT"]
#keys = ["data_ambient_temperature","data_input1_frequency","data_input1_frequency_to_moisture","data_Input2_voltage","data_Input2_voltage_to_temp","data_light_intensity","data_relative_humidity"]
keys = ["data_soil_moisture1","data_soil_moisture2","data_soil_moisture3","data_soil_moisture4","data_tem1","data_tem2","data_tem3","data_tem4","data_tem5","data_tem6","data_tem7"]

outputCSV(devices)

Server:  https://things.iot.ag.purdue.edu:8080
Token: eyJhbGciOiJIUzUxMiJ9.eyJzdWIiOiJ5YW5nMjMwOUBwdXJkdWUuZWR1IiwidXNlcklkIjoiNjRlOWZjYjAtZjc0ZS0xMWVlLWIzYmMtN2ZlNjliZjhkNDExIiwic2NvcGVzIjpbIkNVU1RPTUVSX1VTRVIiXSwic2Vzc2lvbklkIjoiM2I5NjY2YTQtOGM2OC00YjliLWEwYWMtNzY5MDhkNWNkNWRjIiwiaXNzIjoidGhpbmdzYm9hcmQuaW8iLCJpYXQiOjE3MTI4NDY1MjIsImV4cCI6MTcxMjg1NTUyMiwiZmlyc3ROYW1lIjoiRG9uZ3Nlb2siLCJsYXN0TmFtZSI6IllhbmciLCJlbmFibGVkIjp0cnVlLCJpc1B1YmxpYyI6ZmFsc2UsInRlbmFudElkIjoiYWFjNjU1YTAtYWM2Mi0xMWVjLWFiYzgtMWYxYzA5NTgwZTY3IiwiY3VzdG9tZXJJZCI6Ijc1NzZiMDIwLWVjYWUtMTFlYy1iNzJiLTVkZDc2Y2E1MmEyYiJ9.N0PmsYDLhArorybDQkqWSBevXFMWZRngqi9C4pA48sav5Gs9nvr9cpiBZhviAIJFaEzVyRbNaPNLyXpdwwrRpg
2024-02-11 18:00:00 2024-04-11 06:00:00
1707674400000 1712815200000
[['137feb20-0ea0-11ed-ba72-9f81aa692cc5',
  'ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-2491',
  'a84041315185231b'],
 ['1b7224a0-ecaf-11ec-b72b-5dd76ca52a2b',
  'ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-0113',
  'a840415d3184c7c9'],
 ['3db697b0-0934-11ee-a944-3b

# Initial data reading process
***
## readraw_data Function

The `readraw_data` function is used to read raw data from a CSV file and parse it into a pandas DataFrame.

### Parameters

- `destination`: The path where the output file will be saved.
- `filename`: The name of the CSV file to be read.
- `foutname`: The name of the output file.
- `st_date`: The start date for the data to be read.
- `ed_date`: The end date for the data to be read.

### Returns

- `raw_data`: A pandas DataFrame that contains the data read from the CSV file.

### Functionality

The function works by using the pandas `read_csv` function to read the CSV file. It specifies the delimiter as ';' and parses the 'Timestamp' column as dates. It also specifies the data types for the soil moisture columns to be float64.

The function then returns the DataFrame.
***
## station_data_clean Function

The `station_data_clean` function is used to clean the data for a specific station.

### Parameters

- `raw_data`: A pandas DataFrame that contains the raw data to be cleaned.
- `station`: An integer that represents the station ID.

### Returns

- `station_data`: A pandas DataFrame that contains the cleaned data for the specified station.

### Functionality

The function works by filtering the raw data for the specified station. It then performs any necessary cleaning operations, such as removing missing values, outliers, or incorrect data.

The function then returns the cleaned data for the specified station.

Please note that the actual code for the `station_data_clean` function is not provided, so the parameters and functionality are assumed based on typical usage. If you provide the actual code of the `station_data_clean` function, I can give a more accurate explanation.

In [2]:
# -*- coding: utf-8 -*-
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import datetime as dt
import statistics as stats
import os


today = dt.date.today()
# today = datetime(2009, 7, 6, 0, 0)
strtoday = today.strftime("%Y%m%d")
print ('Today is :: ',today)



def readraw_data(destination, filename, foutname, st_date, ed_date):
    
    # open the file
    raw_data = pd.read_csv(filename,delimiter=';', parse_dates=['Timestamp'],
                          dtype={'data_soil_moisture1':np.float64,
                                 'data_soil_moisture2':np.float64,
                                 'data_soil_moisture3':np.float64,
                                 'data_soil_moisture4':np.float64},
                          na_values=['Invalid data']
                          )
    raw_columns = raw_data.columns.tolist()
    # raw_data['Timestamp'] = pd.to_datetime(raw_data['Timestamp'])
    # drop the temperature data from the list
    for i in range(len(raw_columns)):
        if 'tem' in raw_columns[i] or 'dev' in raw_columns[i]:
            raw_data = raw_data.drop(columns=raw_columns[i])
            
    raw_data.set_index(['Timestamp'])
    print(raw_data.info())
    
    # change the name of the Entity Name column
    raw_data['Entity Name'] = raw_data['Entity Name'].str.replace('ABE-DRAGINO-GROPOINT-CHERKHAUER-ACRE-','')
    stationlist = sorted(raw_data['Entity Name'].unique())
        
    
    # after checking the null values, filter the data
    clean_df = pd.DataFrame(columns = ['Station','Layer1', 'Layer2', 'Layer3', 'Layer4'])
    for station in stationlist:
        # clean the data according to the cleaning procedure
        sample_df = station_data_clean(destination, raw_data, station)
        
        clean_df = pd.concat([clean_df,sample_df])
        
    clean_df.to_csv(destination+foutname,index=True)
    
    # convert headers only with numbers
    raw_columns = raw_data.columns.tolist()
    
    # group the dataframe and turn them into another dataframe with 'Entity Name' as columns.
    # this step should be done by layer.
    

    return raw_data, raw_columns, clean_df, stationlist, raw_columns



# ********************************************************************************************************************
# * this is the function that goes into readraw_data function.                                                       *
# ********************************************************************************************************************
def station_data_clean(destination, raw_data, station):
    global start_date, end_date
    '''
    This is cleaning for station data

    Parameters
    ----------
    raw_data : dataframe
        dataframe of the raw_data, 'raw_data' in this script
    station : string
        this is number of station. 4 digit number filled with zero from left

    Returns
    -------
    sample_df : dataframe
        this cleaned data after process

    '''
    sample_df = raw_data[raw_data['Entity Name']==station]
    sample_df = sample_df.drop(['Entity Name'], axis=1)
    sample_df = sample_df.set_index('Timestamp')
    sample_df = sample_df.set_axis(['Layer1', 'Layer2', 'Layer3', 'Layer4'], axis=1)
    
    # clean the data by time of interest
    # datetime range should start by 18:00 // end by 6:00 for better analysis for everyday
    start_date = dt.datetime(st_date[0], st_date[1], st_date[2], 0, 0, 0)
    end_date = dt.datetime(ed_date[0], ed_date[1], ed_date[2], 6, 0, 0)
    sample_df = sample_df[(sample_df.index > start_date) & (sample_df.index < end_date)]
    
    # according to the number of the data length,
    # if it is more than 1, the dataframe will be made
    # if it is 0, below process will be skipped
    totnum = len(sample_df)
    if totnum > 0:    
        print ('\n\n')
        txt = ' raw_data info for station '+station+' '
        print(txt.center(60,'='),end='\n')
        
        # if value is null value from beginning
        print(' NaN values info '.center(60,':'))
        na_df = sample_df[sample_df.isna().any(axis=1)]
        sample_df = sample_df.dropna()
        print('NaN values are ::',len(na_df),'out of',totnum,'\nerror rate:',round(len(na_df)/totnum*100,2),'%',end='\n\n')
        
        # if value is out of range
        sample_df[(sample_df>=100.0) | (sample_df<=0.0)] = np.nan
        outrange_df = sample_df[sample_df.isna().any(axis=1)]
        sample_df = sample_df.dropna()
        print('Out of range values are ::',len(outrange_df),'out of',totnum,'\nerror rate:',round(len(outrange_df)/totnum*100,2),'%',end='\n\n')
        
        print(' Data Describe '.center(60,':'),end='\n\n')
        print(sample_df.dtypes, end='\n\n')
        print(sample_df.describe())
        
        # resampling in 30 min frequency
        sample_df = sample_df.resample('30min').mean()
        
        # make boxplot per layer
        boxplot = sample_df.boxplot(column=['Layer1', 'Layer2', 'Layer3'],figsize=(8,4), ylabel='Soil Moisture (%)')
        plt.title(int(station))
        plt.savefig(destination+'station_data_'+station+'.png',dpi=600)
        plt.show()
        
        # add station code back
        sample_df['Station'] = int(station)
    
        print('='*60)
        
    else:
        txt = station+' '
        print(txt.center(60,'='),end='\n\n')
    '''
    # save station dataframe as csv file == turned off for cal_deficit
    try:
        sample_df.to_csv(destination+'station_data/'+station+'_cleaned.csv',sep=',')
        print('Transmitter data exported.')
    except:
        print('Transmitter data export failed.')
    '''
    
    return sample_df

ModuleNotFoundError: No module named 'matplotlib'

## Rainfinder Function

The `rainfinder` function is used to identify significant rain events in a given dataset. The function is designed to analyze weather data and detect periods of rainfall based on certain conditions or thresholds.

### Parameters

- `data`: A pandas DataFrame that contains the weather data to be analyzed. The DataFrame should be indexed by 'Timestamp and Station'.
- `station`: An integer that represents the station ID.
- `header`: A list that contains the headers of the data.
- `threshold_moist`: An integer that represents the threshold of soil moisture difference to decide if it was a significant rain event or not.
- `raintimestep`: An integer that represents the number of time steps to consider for the rolling window.

### Returns

- `bumplist2`: A list of lists. Each inner list represents a date (in the format [year, month, day]) when a significant rain event (or "bump") was detected.

### Functionality

The function works by first filtering the data for the specified station and removing any missing values. It then calculates the difference in soil moisture between each time step and filters out the time steps where the difference is greater than the specified threshold.

The function then uses a rolling window to find the minimum and maximum soil moisture values within each window. It calculates the gap between the min and max values and filters out the time steps where the gap is greater than the threshold.

The function then organizes the dates of these significant rain events, removes duplicates, and sorts the list. The sorted list of dates is returned.

The function uses pandas for data manipulation and filtering.

In [9]:
def rainfinder(data, station, header, threshold_moist, raintimestep):
    """
    

    Parameters
    ----------
    data : dataframe
        organized dataframe / index is 'Timestamp and Station'
    stationlist : list
        list of the stations
    header : list
        header of the data
    threshold_moist : int
        threshold of soil moisture difference to decide it was huge rain event or not
        
    Returns
    -------
    data_dur_all : list of dataframe
        this is dataframes with time range of interest
        dataframe order is accordance with order of station list (stationlist)

    """
    # TODO: remove stationlist for the final version
    text = station +' :: '+target+'  Rainfinder'
    print (text.center(60,':'))

    data_dur = data[data['Station']==int(station)]
    data_dur = data_dur.dropna()
    
        
    # make a rolling window column from interpolation
    # data_dur[target] = data_dur[target].interpolate()
    print (station,'|','LEN =', len(data_dur))
    
    # find differene between each time step
    temp = data_dur[target].diff()
    filtered_temp = temp.to_frame(name=target).query('{target} > @threshold_moist'.format(target = target))
    
    # find biggest difference within 3 hours (6 timesteps) == bump
    data_dur['min_'+target] = data_dur[target].rolling(window=raintimestep).min()
    data_dur['max_'+target] = data_dur[target].rolling(window=raintimestep).max()
    data_dur['gap_'+target] = data_dur['max_'+target] - data_dur['min_'+target]
    filtered_temp = data_dur.query('gap_{target} > @threshold_moist'.format(target = target))
    #print (filtered_temp)

    # organizing the date of bump
    bumplist = filtered_temp.index.to_list()        
    for i in range(len(bumplist)):
        # make a list of date with the bump in the form of [year, month, day]
        bumplist[i] = [bumplist[i].year, bumplist[i].month, bumplist[i].day]
    # remove duplicates in bumplist
    bumplist2 = list(set(map(tuple, bumplist)))
    # sort the list and convert tuples into list
    bumplist2 = sorted(bumplist2)
    for i in range(len(bumplist2)):
        bumplist2[i] = list(bumplist2[i])

    print ('Searched ', len(bumplist2), 'days with bump events')
    print (bumplist2)

    return bumplist2



## FCfinder Function

The `FCfinder` function is used to find the field capacity of soil after a given number of days from a specified start date. Field capacity is the amount of soil moisture or water content held in the soil after excess water has drained away and the rate of downward movement has decreased. This capacity is reached within 2–3 days after rain or irrigation in typical soil conditions.

### Parameters

- `data`: A pandas DataFrame that contains the data to be analyzed. The DataFrame should be indexed by 'Timestamp and Station'.
- `station`: An integer that represents the station ID.
- `header`: A list that contains the headers of the data.
- `start_date`: A list that contains the start date of the data search in the format [year, month, day].
- `search_days`: An integer that represents the number of days to search for field capacity.
- `search_range`: An integer that represents the range (in hours) for moving field capacity.
- `search_slope`: A float that represents the slope for searching.
- `threshold_hour`: An integer that represents the threshold hours that remains flat soil moisture behavior to confirm it is a field capacity point.

### Returns

- `data_dur_all`: A list of pandas DataFrames. Each DataFrame contains a time range of interest. The order of the DataFrames corresponds to the order of the station list.

### Functionality

The function works by iterating over a specified number of days from the start date. For each day, it identifies a time range around midnight and checks if the standard deviation of the soil moisture within this time range is less than the specified slope. If it is, the function considers this as a constant moisture behavior and records the date and the mean soil moisture value. The function then plots the soil moisture and the field capacity points, saves the plot as a PNG file, and writes the field capacity records to a CSV file.

The function uses a rolling window to smooth the soil moisture data and uses the standard deviation to identify constant moisture behavior. It also uses matplotlib for plotting and os for file operations.

In [10]:
def FCfinder(data, station, header, start_date, search_days, search_range, search_slope, threshold_hour):
    global searched, precip_search1, precip_search2, precip_delta, precip_max

    # print ('Station',station, 'Date',start_date)
    # important == rl means rolling window. if you don't want, remove it.
    search_target = 'rl_'+target

    data_dur_all = []

    #data.plot(x = data.index,  y = target, data = data[data['Station']==station])

    data_dur = data[data['Station']==int(station)]
    
        
    # make a rolling window column from interpolation
    data_dur[target] = data_dur[target].interpolate()
    # print (data_dur[target])
    data_dur['rl_'+target] = data_dur[target].rolling(window=3).mean()
    

    '''
    From here, we need to find out point where soil moisture value is consistent for 2 hours (threshold_hour)
    And consistent variation for this hour is < 1% (search_slope)
    Searching range will be +- 4hrs from midnight (search_range)
    '''
    # set the database only for the search_target
    target_data = data_dur[search_target]
    
    # find 00:00 hour of everyday within TOI (7 days)
    dayrange = search_days
    
    # this list is for the searched fc dates
    searched_list = []
    
    # searching field capacity (daily)
    for days in range(1,dayrange+1):
        # Going to find the field capacity after given days from the start_date
        st_datetime = dt.datetime(start_date[0], start_date[1], start_date[2], 6, 0, 0)

        # stamp2 is 4 hours before the midnight of the stamp1
        # stamp3 is 4 hours after the midnight of the stamp2
        # add 'days' to search every daily step
        stamp1 = st_datetime + dt.timedelta(days=days)
        stamp1 = stamp1.replace(hour=0, minute=0, second=0)
        # print(stamp1)
        
        # step 1. select the time range of interest
        # once you find the time, you will search there is constant soil moisture behavior or not
        # set the searching time
        stamp2 = stamp1 - dt.timedelta(hours=search_range)
        stamp3 = stamp1 + dt.timedelta(hours=search_range)
        # print(' SM ',stamp2, '--' ,stamp3)
        
        # step 2. set the searching range
        searched = target_data[(target_data.index > stamp2) & (target_data.index < stamp3)]
        # searched = data_search
            
        try:
            # print('STD::',round(stats.stdev(searched),2), 'MEAN::',round(stats.mean(searched),2))
            
            # if standard deviation does not exceeds 'search_slope',
            # this will be regarded as constant moisture behavior
            if stats.stdev(searched) < search_slope:
                searched_list.append([stamp1, stats.mean(searched)])
                FC_searched = True
            else:
                # print(stamp2,'has no moving field capacity.', end='\n\n')
                FC_searched = False
        except:
            # print(station, stamp2, 'has no data.', end='\n\n')
            FC_searched = False
        
        if FC_searched == True:
            print ('Field Capacity is found for',str(start_date[0])+'-'+str(start_date[1])+'-'+str(start_date[2]),':',searched_list[0][0].strftime('%Y-%m-%d'),round(searched_list[0][1],3))

        
            # list of searched field capacity
            searched_dates = [i[0] for i in searched_list]
            searched_values = [i[1] for i in searched_list]
            
            # if field capacity is found, draw graph
            if len(searched_dates) > 0:
                # draw overlaying graph for the soil moisture and field capacity
                # soil moisture is blue line and field capacity is red dot            
                # data_dur = data_dur.reset_index()
                df_subset = data_dur[(data_dur.index < stamp1+dt.timedelta(days=1)) & (data_dur.index > st_datetime)]
                print(df_subset)

                fig, ax = plt.subplots(figsize=(12, 5))
                # ax = data_dur.plot(y=target, label = 'Observed values', figsize=(15,5))
                ax = plt.plot(df_subset.index, df_subset[target], label = 'Observed values')

                # ax.set_xlim(stamp2, stamp3)
                plt.scatter(x=searched_dates, y=searched_values, label = 'Moving FC points', marker='s', c='r', s=100)
                plt.legend()
                plt.gcf().autofmt_xdate()
                plt.xlabel('Date')
                plt.ylabel('Soil Moisture (%)')
                plt.title('Field Capacity for '+str(station)+' after '+st_datetime.strftime('%Y-%m-%d'))

                # save graph for the soil moisture and field capacity
                graph_dest = os.getcwd()+'/fc_graphs/'+st_datetime.strftime('%Y-%m-%d')
                if not os.path.exists(graph_dest):
                    os.makedirs(graph_dest)
                plt.savefig(graph_dest+'/'+str(station)+'_'+stamp1.strftime('%Y-%m-%d')+'.png',dpi=600)
                plt.show()
                print('Graph is saved.')
            
            # save field capacity records in csv form
            data_dur_all.append(data_dur)
            
            
            # save field capacity records in csv form
            with  open(os.getcwd()+'/'+'fc_record.csv', 'a+') as f:
                for item in searched_list:
                    f.write(str(station)+','+item[0].strftime('%Y-%m-%d')+','+str(round(item[1],2))+'\n')
            
            break
    
    return data_dur_all







def FCfinder2(data, station, header, start_date, search_days, search_range, search_slope, threshold_hour):
    global searched, precip_search1, precip_search2, precip_delta, precip_max
    """
    This function finds the field capacity of the soil given a dataset, station, header, start date, 
    number of search days, search range, search slope, and threshold hour.
    """
    
    # Target variable
    target = header
    
    # Interpolating missing values and creating a rolling window column
    data_dur = data[data['Station'] == int(station)]
    data_dur[target] = data_dur[target].interpolate()
    data_dur['rl_' + target] = data_dur[target].rolling(window=3).mean()
    
    # Defining search target
    search_target = 'rl_' + target
    
    # Data duration list
    data_dur_all = []
    
    # Searching field capacity for each day in the range
    for days in range(1, search_days + 1):
        st_datetime = dt.datetime(start_date[0], start_date[1], start_date[2], 6, 0, 0)
        stamp1 = st_datetime + dt.timedelta(days=days)
        stamp1 = stamp1.replace(hour=0, minute=0, second=0)
        
        # Setting search range
        stamp2 = stamp1 - dt.timedelta(hours=search_range)
        stamp3 = stamp1 + dt.timedelta(hours=search_range)
        searched = data_dur[(data_dur.index > stamp2) & (data_dur.index < stamp3)]
        
        try:
            # Checking for constant moisture behavior
            if stats.stdev(searched) < search_slope:
                searched_list.append([stamp1, stats.mean(searched)])
                FC_searched = True
            else:
                FC_searched = False
        except:
            FC_searched = False
        
        if FC_searched:
            print('Field Capacity is found for', str(start_date[0]) + '-' + str(start_date[1]) + '-' + str(start_date[2]), ':', searched_list[0][0].strftime('%Y-%m-%d'), round(searched_list[0][1], 3))
        
            # Graphing
            searched_dates = [i[0] for i in searched_list]
            searched_values = [i[1] for i in searched_list]
            
            if len(searched_dates) > 0:
                df_subset = data_dur[(data_dur.index < stamp1 + dt.timedelta(days=1)) & (data_dur.index > st_datetime)]
                fig, ax = plt.subplots(figsize=(12, 5))
                ax = plt.plot(df_subset.index, df_subset[target], label='Observed values')
                plt.scatter(x=searched_dates, y=searched_values, label='Moving FC points', marker='s', c='r', s=100)
                plt.legend()
                plt.gcf().autofmt_xdate()
                plt.xlabel('Date')
                plt.ylabel('Soil Moisture (%)')
                plt.title('Field Capacity for ' + str(station) + ' after ' + st_datetime.strftime('%Y-%m-%d'))
                
                graph_dest = os.getcwd() + '/fc_graphs/' + st_datetime.strftime('%Y-%m-%d')
                if not os.path.exists(graph_dest):
                    os.makedirs(graph_dest)
                plt.savefig(graph_dest + '/' + str(station) + '_' + stamp1.strftime('%Y-%m-%d') + '.png', dpi=600)
                plt.show()
                print('Graph is saved.')
            
            data_dur_all.append(data_dur)
            
            # Saving field capacity records in CSV form
            with open(os.getcwd() + '/' + 'fc_record.csv', 'a+') as f:
                for item in searched_list:
                    f.write(str(station) + ',' + item[0].strftime('%Y-%m-%d') + ',' + str(round(item[1], 2)) + '\n')
            
            break
    
    return data_dur_all


# About the 'Defecit Calc' function
## About
***


# Below is implementation Block

In [11]:
import warnings
warnings.filterwarnings('ignore')

'''
Implementation Block

set up the variables for running the programs
you will set the destination of the files (directory)
you will set the date of your interest.
usually, it is set to today's date

'''

destination = ''
filename = destination + 'Raw_data/Calc_def_test.csv'
# ! filename should be changed to the file name of the downloaded data
foutname = 'FC_data'+strtoday+'.csv'

# set the time of interest and location [year, month, day]
#  this is for the data of soil moisture. = for the date of the analysis
# for regular 
ed_date = [dt.date.today().year, dt.date.today().month, dt.date.today().day]
# end date will be 
st_datetime = dt.datetime(ed_date[0], ed_date[1], ed_date[2], 6, 0, 0) - dt.timedelta(days=360)
st_date = [st_datetime.year, st_datetime.month, st_datetime.day]

raw_data, raw_columns, clean_df, stationlist, raw_columns = readraw_data(destination, filename, foutname, st_date, ed_date)


''' 
This part should be done per station [stationlist]
'''
# stationlist = ['0111','0112','0114']
print ('Station List ::',stationlist)

# this is the target layer for the analysis. 1=layer 1
# variable set for rainfinder
target = 'Layer2'
threshold_moist = 15    # threshold of soil moisture difference to decide it was huge rain event or not
raintimestep = 12 # 1 = 1/2 hour // 12 = 6 hours

# variable set for FCfinder
search_days = 14 # days
search_range = 5 # hours
search_slope = 0.07 # unit is fraction for standard deviation of the soil moisture behavior
threshold_hour = 4 # hours with consistent soil moisture behavior


for station in stationlist:
    print('\n\n')
    text = station +' :: '+target
    print (text.center(60,'='))
    bumplist = rainfinder(clean_df, station, raw_columns, threshold_moist, raintimestep)
    for start_date in bumplist:
        FCfinder2(clean_df, station, raw_columns, start_date, search_days, search_range, search_slope, threshold_hour)




# data_dur_all = FCfinder(clean_df, stationlist, raw_columns, search_range, search_slope, threshold_hour)

NameError: name 'strtoday' is not defined

# About the 'Defecit calc' function
## About
***
### Job in the script
This is for calcuating soil moisture defecit

In [ ]:
# root depth
root=24

df2.loc[df2['soil_moisture1'] < fcdf.fc1, 'd1']=fcdf.fc1-df2['soil_moisture1']
df2.loc[df2['soil_moisture2'] < fcdf.fc2, 'd2']=fcdf.fc2-df2['soil_moisture2']
df2.loc[df2['soil_moisture3'] < fcdf.fc3, 'd3']=fcdf.fc3-df2['soil_moisture3']
df2.loc[df2['soil_moisture4'] < fcdf.fc4, 'd4']=fcdf.fc4-df2['soil_moisture4']

if root < 6:
    df2['Deficit'] = (df2['d1'])*root
elif root < 12:
    df2['Deficit'] = (df2['d1'])*6 + (df2['d2'])*(root-6)
elif root < 18:
    df2['Deficit'] = (df2['d1'])*6 + (df2['d2'])*6 + (df2['d3'])*(root-12)
else:
    df2['Deficit'] = (df2['d1'])*6 + (df2['d2'])*6 + (df2['d3'])*6 + (df2['d4'])*(root-18)




